In [232]:
import numpy as np

## Task 1
Implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the following substitution function:
    $$f : y_j(j): 2v_i(j) \mod  p, \ j$$

In [233]:
def subkey_generator_default(key):

    k_1 = [key[0],key[2],key[4],key[6]] 
    k_2 = [key[0],key[1],key[2],key[3]] 
    k_3 = [key[0],key[3],key[4],key[7]] 
    k_4 = [key[0],key[3],key[5],key[6]] 
    k_5 = [key[0],key[2],key[5],key[7]] 
    k_6 = [key[2],key[3],key[4],key[5]] 

    return [k_1, k_2, k_3, k_4, k_5, k_6]

def subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter)

    return (w + k)%p

In [234]:
 
def timesTwo(v,p):

    return np.multiply(v,2)%p

def transposition(y):

    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])

def linear(z, p):

    z_matrix = z.reshape(2,4)

    param_matrix = np.matrix([[2,5],[1,7]])
    
    w_matrix = np.dot(param_matrix,z_matrix)%p

    w_array = np.asarray(w_matrix).flatten()
    return w_array
        




In [235]:
def encryption(u,k,p,round,substitution,transposition,subkey_sum,linear, subkey_generator):

    k_list = subkey_generator(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)         
        #print("post subkey_sum",v)
        y = substitution(v,p)
        #print("post timesTwo",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x


In [236]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p=11
r = 5

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

print(x)


[4 0 0 9 7 0 0 3]


## Task 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field $\mathbb{F} = GF(p)$.

In [237]:
def inv_timesTwo(v,p):
    mul_inv = pow(2,-1,p)
    return np.multiply(v,mul_inv)%p


def inv_transposition(y):
    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])


def inv_subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter) #list

    return (w - k)%p

#Calcolo dell'inversa della matrice modulo p
def inv_matrix(m, p):
    det = round(np.linalg.det(m))
    m_ast = np.linalg.inv(m)

    m_tilda = m_ast*det

    det_inv = pow(det, -1, p)

    m_inv = np.round(m_tilda * det_inv).astype(int) % p

    return m_inv

def inv_linear(z, p):
    z_matrix = z.reshape(2, 4)

    param_matrix = np.array([[2, 5], [1, 7]])

    #Calcolo dell'inversa della matrice modulo p
    param_matrix_inv = inv_matrix(param_matrix, p)

    w_matrix = np.dot(param_matrix_inv, z_matrix) % p

    w_array = np.asarray(w_matrix, dtype=int).flatten()

    return w_array


def decryption(x,k,p,round, inv_substitution, inv_transposition, inv_subkey_sum, inv_linear, subkey_generator):
    k_list = subkey_generator(k)

    for i in range(round,-1,-1):
        #print("round : ", i)
        z = inv_subkey_sum(k_list[i],x,p)            #v = u - k
        #print("post subkey_sum",z)
        if i == 0:
            x = z
            break
        elif i != round:
            w = inv_linear(z, p)
            #print("post linear", w)
        else:
            w = z
        y = inv_transposition(w)
        #print("post transpo",y)
        x = inv_substitution(y,p)
        #print("post timesTwo",x)
        
    return x

Test for inv_linear

In [238]:
a = linear(np.array([1,2,3,5,4,6,4,8]), p)  #change the array to test
print(a)
b = inv_linear(a, p)
print(b)

[0 1 4 6 7 0 9 6]
[1 2 3 5 4 6 4 8]


In [240]:
u = np.array([1,0,0,0,0,0,0,0])
x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)
                
u = decryption(x,k,p,r, 
               inv_substitution=inv_timesTwo,
               inv_transposition=inv_transposition,
               inv_subkey_sum=inv_subkey_sum,
               inv_linear=inv_linear,
               subkey_generator=subkey_generator_default)
print(u)

[1 0 0 0 0 0 0 0]


Other tests for decryption

In [242]:
a = np.array([1,8,3,6,8,6,9,10]) #change plaintext to test
b = np.array([1,0,4,0,3,0,2,0]) #change key to test
c = encryption(a,b,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

print(c)
d = decryption(c,b,p,r, 
               inv_substitution=inv_timesTwo,
               inv_transposition=inv_transposition,
               inv_subkey_sum=inv_subkey_sum,
               inv_linear=inv_linear,
               subkey_generator=subkey_generator_default)
print(d)

[ 8  6  1 10  4  8 10  2]
[ 1  8  3  6  8  6  9 10]


## Task 3
Identify the overall linear relationship for this simplified AES-like cipher, that is find the
matrices $A ∈ F^{(ℓ_x × ℓ_k)}$ and $B ∈ F^{(ℓ_x × ℓ_u)}$ such that
$$x = E(k, u) = Ak + Bu \mod p$$
with all operations in the field $\mathbb{F} = GF(p)$.

In [186]:
def linear_relationship():  #pass the encryption function as a parameter
    k_a = np.eye(8, dtype=int)
    e_a = np.zeros(8, dtype=int)
    A = np.empty((8,0), dtype=int)

    e_b = np.eye(8, dtype=int)
    k_b = np.zeros(8, dtype=int)
    B = np.empty((8,0), dtype=int)

    #k = e_j, u = 0
    for i in range(8):

        encrypted_value = encryption(e_a, np.asarray(k_a[i]), p, r,
                                    substitution=timesTwo,
                                    transposition=transposition,
                                    subkey_sum=subkey_sum,
                                    linear=linear,
                                    subkey_generator=subkey_generator_default)
        A = np.hstack((A, encrypted_value.reshape(-1, 1)))
        #print(encrypted_value)
        #print(k_a[i])
    #print(A)

    for i in range(8):

        encrypted_value = encryption(np.asarray(e_b[i]),k_b,p,r,
                                    substitution=timesTwo,
                                    transposition=transposition,
                                    subkey_sum=subkey_sum,
                                    linear=linear,
                                    subkey_generator=subkey_generator_default)
        B = np.hstack((B, encrypted_value.reshape(-1, 1)))


    #print(B)

    return A, B

In [187]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

A, B = linear_relationship()

#print(A)
#print(B)
print((np.dot(A,k) + np.dot(B,u))%p)
print(np.all(x == (np.dot(A,k) + np.dot(B,u))%p))

[4 0 0 9 7 0 0 3]
True


In [188]:
u_1 = np.array([1,2,0,4,0,0,4,0]) #change plaintext to test
k_1 = np.array([0,0,0,0,0,0,0,1]) #change key to test

x = encryption(u_1,k_1,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

A, B = linear_relationship()

print(A)
print(B)

print(x)
print((np.dot(A,k_1) + np.dot(B,u_1))%p)

print(x == (np.dot(A,k_1) + np.dot(B,u_1))%p)

[[ 9  0  1  6  0  0  1 10]
 [ 0  8  6  2  2  9  0  0]
 [ 0  6  0  8  3 10  0  0]
 [ 6  0  0  8  0  1  6  6]
 [ 2  0  1 10  0  0  1  3]
 [ 0  1  8  4  9  6  0  0]
 [ 0 10  0  5  7  6  0  0]
 [ 3  0  0  1  0  1  4  8]]
[[6 0 0 3 3 0 0 0]
 [0 6 3 0 0 3 0 0]
 [0 3 6 0 0 0 3 0]
 [3 0 0 6 0 0 0 3]
 [5 0 0 0 4 0 0 8]
 [0 5 0 0 0 4 8 0]
 [0 0 5 0 0 8 4 0]
 [0 0 0 5 8 0 0 4]]
[6 1 7 0 8 9 5 6]
[6 1 7 0 8 9 5 6]
[ True  True  True  True  True  True  True  True]


## Task 4
From a known plaintext/ciphertext pair $(u, x)$, implement a linear cryptanalysis KPA against this cipher by computing 
$$k = A^{−1}(x − Bu) \mod p$$ 
with all operations in the field $\mathbb{F} = GF(p)$

In [189]:
def linear_KPA(A,B,u,x):
    A_inv = inv_matrix(A, p)
    k = np.dot(A_inv, (x - np.dot(B,u))) % p
    return k

In [190]:
A,B = linear_relationship()
u = np.array([1,0,0,7,0,0,0,0]) #change plaintext to test
k = np.array([1,0,0,3,0,0,5,7]) #change key to test
x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear = linear,
               subkey_generator=subkey_generator_default)

print(x)
key = linear_KPA(A,B,u,x)

print(key)


[8 6 2 4 8 1 4 7]
[1 0 0 3 0 0 5 7]


In [191]:
import ast
A, B = linear_relationship()
with open('KPAdataQ/KPApairsQ_linear.txt', 'r') as file_in:
    with open('KPAdataQ/KPAkeysQ_linear.txt', 'w') as file_out:
        first = True
        correct = True
        for line in file_in:
            u_str, x_str = line.split('\t')
            u = np.array(ast.literal_eval(u_str.strip()))
            x = np.array(ast.literal_eval(x_str.strip()))
            print(u, x)
            k = linear_KPA(A, B, u, x)
            if first:
                k_first = k
                first = False
            elif not np.array_equal(k, k_first):
                print("**ERROR: Different k**")
                correct = False
                break
        if correct:
            print("**SUCCESS: All k are the same**")
            print(k)
            file_out.write(str(k.tolist()) + '\n')

[5 9 9 9 2 5 9 1] [ 2 10  8  9 10  0 10  7]
[9 9 2 0 5 9 5 4] [8 1 9 9 0 6 2 9]
[5 1 1 9 6 2 9 0] [3 6 2 6 7 3 1 2]
[5 7 3 4 5 9 8 0] [7 3 7 9 3 9 8 2]
[3 4 9 4 2 6 5 9] [8 5 3 8 9 2 2 3]
**SUCCESS: All k are the same**
[ 9  1  4  3 10  6  2  1]


## Task 5
implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the substitution function described by the following table
|$v_i(j)$|0|1|2|3|4|5|6|7|8|9|10|
|--------|-|-|-|-|-|-|-|-|-|-|--|
|$y_i(j)$|0|2|4|8|6|10|1|3|5|7|9|

where $j ∈ \{1, . . . , ℓ\}$

In [192]:
def custom_substitution_number(v):
    if(v == 0):
        return 0
    if(v == 1):
        return 2
    if(v == 2):
        return 4
    if(v == 3):
        return 8
    if(v == 4):
        return 6
    if(v == 5):
        return 10
    if(v == 6):
        return 1
    if(v == 7):
        return 3
    if(v == 8):
        return 5
    if(v == 9):
        return 7
    if(v == 10):
        return 9
    
def custom_substitution(v,p):

    for i in range(8):
        v[i] = custom_substitution_number(v[i])
    
    return v


In [193]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p=11
r = 5

print(encryption(u,k,p,r,
                 substitution=custom_substitution,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator=subkey_generator_default
                 ))

[9 0 0 0 5 0 0 6]


## Task 6
Linear cryptanalysis of a “nearly linear” cipher

In [223]:
def generate_data(n_samples=10000, p=11, vector_size=8):
    data = []
    for _ in range(n_samples):
        k = np.random.randint(0, p, vector_size)
        u = np.random.randint(0, p, vector_size)
        x = encryption(u,k,p,r,
                substitution=custom_substitution,
                transposition=transposition,
                subkey_sum=subkey_sum,
                linear=linear,
                subkey_generator=subkey_generator_default
                )
        data.append((k, u, x))
    return data


def random_vectors(dim=8, p=11, max_tries=100):
    for _ in range(max_tries):
        vec = np.random.randint(0, p, dim)
        if np.any(vec != 0):
            yield vec


def evaluate_triplet(data, a, b, c, p=11):
    count = 0
    for k, u, x in data:
        s = (np.dot(a, k) + np.dot(b, u) + np.dot(c, x)) % p
        if s == 0:
            count += 1
    return count / len(data)


def linear_approximation(data,p=11, max_relations=8, bias_thresh=0.2):
    relations = []
    tested = 0
    for a in random_vectors():
        for b in random_vectors():
            for c in random_vectors():
                est_prob = evaluate_triplet(data, a, b, c)
                if est_prob > (1 / pow(p,len(a)) + bias_thresh):
                    relations.append((a.copy(), b.copy(), c.copy(), est_prob))
                    print(f"✔️ Trovata relazione con bias {est_prob:.3f}")
                    if len(relations) >= max_relations:
                        return relations
                tested += 1
                if tested % 5000 == 0:
                    print(f"Testate {tested} combinazioni...")
    return relations

In [221]:
data = generate_data()

print(data)

[(array([9, 2, 5, 7, 6, 2, 2, 1]), array([9, 0, 6, 2, 5, 3, 8, 3]), array([10,  9,  8,  6,  8,  5,  1,  5])), (array([ 6,  9,  8,  6,  5, 10,  3,  5]), array([3, 2, 4, 2, 7, 4, 0, 7]), array([ 0,  7, 10,  4,  8,  4, 10,  9])), (array([ 8,  7,  6,  7,  3,  4,  4, 10]), array([3, 0, 5, 4, 3, 2, 5, 8]), array([4, 6, 1, 2, 0, 2, 7, 5])), (array([ 3,  4,  3,  9,  8,  6,  5, 10]), array([3, 7, 2, 7, 1, 9, 2, 1]), array([8, 2, 8, 9, 3, 8, 3, 9])), (array([2, 6, 9, 9, 3, 6, 1, 7]), array([ 1,  9,  6,  2, 10,  0,  0,  7]), array([6, 3, 7, 9, 7, 0, 1, 1])), (array([ 8, 10,  6, 10,  5,  0,  5,  0]), array([ 5,  3, 10,  7,  4,  1,  5,  7]), array([ 6,  7,  1, 10, 10,  4, 10,  2])), (array([ 7, 10,  3, 10,  4, 10, 10,  8]), array([1, 5, 9, 0, 3, 2, 2, 4]), array([6, 0, 2, 2, 3, 7, 6, 6])), (array([7, 0, 8, 7, 4, 9, 3, 8]), array([9, 8, 3, 3, 9, 0, 4, 1]), array([1, 5, 3, 8, 4, 1, 9, 9])), (array([5, 3, 9, 2, 3, 9, 3, 2]), array([1, 1, 9, 0, 8, 7, 9, 0]), array([ 8,  8,  1,  2,  1,  2,  3, 10])), (a

In [224]:
relations = linear_approximation(data, bias_thresh=0.2)
for rel in relations:
    print(rel)

A = np.array([a for (a, _, _, _) in relations]) % p
B = np.array([b for (_, b, _, _) in relations]) % p
C = np.array([c for (_, _, c, _) in relations]) % p

print("A", A)
print("B", B)
print("C", C)

u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p = 11
r = 5

x = encryption(u,k,p,r, substitution=custom_substitution,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

result = (np.dot(A,k) + np.dot(B,u) + np.dot(C,x)) % p
print(result)

Testate 5000 combinazioni...
Testate 10000 combinazioni...
Testate 15000 combinazioni...
Testate 20000 combinazioni...
Testate 25000 combinazioni...
Testate 30000 combinazioni...
Testate 35000 combinazioni...
Testate 40000 combinazioni...
Testate 45000 combinazioni...


KeyboardInterrupt: 

In [216]:
def generate_data_B(k=np.array([1,0,0,0,0,0,0,0]),n_samples=10000, p=11, vector_size=8):
    data = []
    for _ in range(n_samples):
        u = np.random.randint(0, p, vector_size)
        x = encryption(u,k,p,r,
                substitution=custom_substitution,
                transposition=transposition,
                subkey_sum=subkey_sum,
                linear=linear,
                subkey_generator=subkey_generator_default
                )
        data.append((u, x))
    return data


def random_vectors_linear_k(dim=8, p=11, max_tries=1000):
    for _ in range(max_tries):
        vec = np.random.randint(0, p, dim)
        if np.any(vec != 0):
            yield vec


def evaluate_triplet_linear_k(data, M, b, c, k = np.array([1,0,0,0,0,0,0,0]), p=11):
    count = 0
    for u, x in data:
        s = (np.dot(b, u) + np.dot(c, (x - np.dot(M,k))%p)) % p
        if s == 0:
            count += 1
    return count / len(data)


def construct_matrix(relations, M, p=11):
    B = np.array([b for (b, _, _) in relations]) % p
    C = np.array([c for (_, c, _) in relations]) % p

    A = np.dot(-C, M) % p
    return A, B, C

def linear_approximation_linear_k(data,p=11, max_relations=8, delta_thresh=0.2):
    relations = []
    tested = 0
    M, _ = linear_relationship()
    
    for b in random_vectors_linear_k():
        for c in random_vectors_linear_k():
            bias = evaluate_triplet_linear_k(data, M, b, c)
            delta = bias - (1 / pow(p, len(b)))
            if delta > delta_thresh:
                relations.append((b.copy(), c.copy(), delta))
                print(f"✔️ Trovata relazione con delta {delta:.3f}")
                if len(relations) >= max_relations:
                    A,B,C = construct_matrix(relations, M)
                    return A, B, C, relations
            tested += 1
            if tested % 5000 == 0:
                print(f"Testate {tested} combinazioni...")
    
    A,B,C = construct_matrix(relations, M)
    return A, B, C, relations

In [218]:
data = generate_data_B()


In [220]:
A, B, C, relations = linear_approximation_linear_k(data, delta_thresh=0.1)
for rel in relations:
    print(rel)

print("A", A)
print("B", B)
print("C", C)
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p = 11
r = 5

x = encryption(u,k,p,r, substitution=custom_substitution,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)
result = (np.dot(A,k) + np.dot(B,u) + np.dot(C,x)) % p
print(result)

KeyboardInterrupt: 

## Task 7
implement the encryptor for a simplified AES-like cipher with the following parameters: $\mathcal{K} = \mathbb{F}^{\ell_k}, \ell_k = 4$

ECC...

In [245]:
def subkey_generator_task_7(key):

    k_1 = [key[0],key[1],key[2],key[3]]
    k_2 = [key[0],key[1],key[3],key[2]]
    k_3 = [key[1],key[2],key[3],key[0]]
    k_4 = [key[0],key[3],key[1],key[2]]
    k_5 = [key[2],key[3],key[0],key[1]]
    k_6 = [key[1],key[3],key[0],key[2]]

    return [k_1,k_2,k_3,k_4,k_5,k_6]

def mod_inv(v, p):
    
    x =[]

    for num in v:
        if(num != 0):
            x.append((2*pow(num, -1, p))%p) # Calcola l'inverso di x modulo p e lo moltiplica per 2
        else:
            x.append(0)


    return np.array(x) 

def substitution_task_7(x,p):
    x= x.tolist()
    return mod_inv(x, p)
    

In [246]:
u= np.array([1,0,0,0,0,0,0,0])
k=np.array([1,0,0,0])

p= 11

print(encryption(u,k,p,r,
                 substitution=substitution_task_7,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator = subkey_generator_task_7))

[5 0 3 2 5 2 1 1]


## Task 8

Implement a “meet-in-the-middle” attack see against the concatenation of two
instances of the non linear simplified AES-like cipher defined in Task 7, with different keys
$k^′, k^{′′}$, respectively.

In [227]:
def double_enc(u, k1, k2, p, r):
    u2 = encryption(u,k1,p,r,
                 substitution=substitution_task_7,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator = subkey_generator_task_7)
    
    x = encryption(u2,k2,p,r,
                 substitution=substitution_task_7,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator = subkey_generator_task_7)
    return x

In [249]:
def inv_substitution_task_7(x,p):
    inv_x = []
    x = x.tolist()
    for elem in x:
        if elem == 0:
            inv_x.append(0)
        elif elem % 2 == 0:
            inv_x.append(pow((elem//2), -1, p))
        else:
            inv_x.append(pow(((elem+11)//2), -1, p))
    return np.array(inv_x)  

Test for `inv_substitution_task_7()`

In [251]:
a = np.array([1,2,3,4,5,6,7,8,9,10])
sub_a = substitution_task_7(a,11)
print(sub_a)
inv_sub_a = inv_substitution_task_7(sub_a,11)
print(inv_sub_a)
print(np.all(a == inv_sub_a)) 

[ 2  1  8  6  7  4  5  3 10  9]
[ 1  2  3  4  5  6  7  8  9 10]
True


test decryption

In [258]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0])
p = 11
r = 5

x = encryption(u,k,p,r,
                 substitution=substitution_task_7,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator = subkey_generator_task_7)

u_test = decryption(x,k,p,r,
                 inv_substitution=inv_substitution_task_7,
                 inv_transposition=inv_transposition,
                 inv_subkey_sum=inv_subkey_sum,
                 inv_linear=inv_linear,
                 subkey_generator = subkey_generator_task_7)

print(u_test)
print(np.all(u == u_test))

[1 0 0 0 0 0 0 0]
True


In [313]:
def generate_unique_random_arrays(n_samples=10000, key_size=4, p=11):
    unique_arrays = set()   #set to not admit duplicates
    while len(unique_arrays) < n_samples:
        #convert to tuple to make it hashable
        new_array = tuple(np.random.randint(0, p, key_size))
        unique_arrays.add(new_array)
    return [np.array(arr) for arr in unique_arrays]


def forward_step(u1,keys, p=11,r = 5):
    fs = []
    for k1 in keys:
        x1 = encryption(u1,k1,p,r,
                 substitution=substitution_task_7,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator = subkey_generator_task_7)
        fs.append((k1, x1))
        fs_sorted = sorted(fs, key=lambda item: tuple(item[1]))
    return fs_sorted


def backward_step(x2, keys, p = 11, r = 5):
    bs = []
    for k2 in keys:
        u2 = decryption(x2,k2,p,r,
                 inv_substitution=inv_substitution_task_7,
                 inv_transposition=inv_transposition,
                 inv_subkey_sum=inv_subkey_sum,
                 inv_linear=inv_linear,
                 subkey_generator = subkey_generator_task_7)
        bs.append((k2, u2))
        bs_sorted = sorted(bs, key=lambda item: tuple(item[1]))
    return bs_sorted


def meet_in_the_middle(u, x, k1s, k2s, p=11, r=5):
    #k1s = generate_unique_random_arrays(n_sample, key_size, p)
    #k2s = generate_unique_random_arrays(n_sample, key_size, p)
    keys = []
    fs = forward_step(u, k1s, p, r)
    bs = backward_step(x, k2s, p, r)

    for k1, x1 in fs:
        for k2, u2 in bs:
            if np.array_equal(x1, u2):
                keys.append((k1, k2))
    return keys

In [315]:
u = np.array([1,0,0,0,0,0,0,0])
k1 = np.array([1,2,3,4])
k2 = np.array([5,6,7,8])
p = 11
r = 5

x = double_enc(u, k1, k2, p, r)
print(x)

k1s = generate_unique_random_arrays()
k2s = generate_unique_random_arrays()

guess_keys = meet_in_the_middle(u, x, k1s, k2s)
print(guess_keys)

[7 7 9 1 6 7 1 7]
[(array([6, 2, 2, 7]), array([6, 4, 5, 1])), (array([0, 5, 6, 4]), array([9, 5, 2, 0]))]


In [318]:
import ast

with open('KPAdataQ/KPApairsQ_non_linear.txt', 'r') as file_in:
    with open('KPAdataQ/KPAkeysQ_non_linear.txt', 'w') as file_out:
        first = True
        correct = True
        #first generate the keys to ensure to find the same keys
        k1s = generate_unique_random_arrays()
        k1s = generate_unique_random_arrays()

        for line in file_in:
            u_str, x_str = line.split('\t')
            u = np.array(ast.literal_eval(u_str.strip()))
            x = np.array(ast.literal_eval(x_str.strip()))
            print(u, x)
            guess_keys = meet_in_the_middle(u, x, k1s,k2s)
            print(guess_keys)
            '''if k1 is None:
                print("**ERROR: No keys found**")
                correct = False
                break
            if first:
                k1_first = k1
                k2_first = k2
                first = False
            #elif not equal keys
            elif not np.array_equal(k1, k_first):
                print("**ERROR: Different keys**")
                print("k1", k1)
                print("k1_first", k1_first)
                print("k2", k2)
                print("k2_first", k2_first)
                correct = False
                break
        if correct:
            print("**SUCCESS: All k are the same**")'''
            file_out.write(str(guess_keys) + '\n')

[ 7 10  7  6  4  6  3  4] [ 8  2  4  1  6 10  3  2]
[(array([0, 7, 2, 4]), array([ 2,  9, 10,  7]))]
[ 4  5  5 10  2  3  6  5] [ 5  8 10  0  6  3  1  4]
[(array([0, 7, 2, 4]), array([ 2,  9, 10,  7]))]
[2 6 9 0 4 0 6 5] [ 8  2  0  1  9  6 10  7]
[(array([ 1,  7, 10,  9]), array([5, 5, 0, 0])), (array([0, 7, 2, 4]), array([ 2,  9, 10,  7]))]
[8 2 6 5 9 7 3 3] [4 9 6 8 2 7 7 3]
[(array([0, 3, 1, 2]), array([8, 4, 2, 8])), (array([0, 7, 2, 4]), array([ 2,  9, 10,  7])), (array([7, 2, 6, 0]), array([5, 6, 4, 3]))]
[ 2 10  3  1  3  8  7  7] [8 8 2 2 5 2 5 0]
[(array([0, 7, 2, 4]), array([ 2,  9, 10,  7]))]


In [316]:
u = np.array([1,0,0,0,0,0,0,0])
k1 = np.array([0, 5, 6, 4])
k2 = np.array([9, 5, 2, 0])
p = 11
r = 5
x = double_enc(u, k1, k2, p, r)
print(x)

[7 7 9 1 6 7 1 7]
